# Terrorismo & Violencia Intrafamiliar

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
PATH_FILE = '/content/drive/MyDrive/Police-Data/Bronce'
OUTPUT_PATH_FILE = '/content/drive/MyDrive/Police-Data/Silver'

## Libraries

In [3]:
import pandas as pd
import numpy as np
import os
import glob

In [4]:
def find_header_row(file_path):
    """
    Finds the header line with a column name
    """
    column_name = ['DEPARTAMENTO','DEPARTAMENTO ','Departamento','departamento']
    header_row = None

    df = pd.read_excel(file_path, header=None)

    for i, row in df.iterrows():
        if column_name[0] in row.values or column_name[1] in row.values or column_name[2] in row.values or column_name[3] in row.values:
            header_row = i
            break
        if i>20:
            break

    if header_row is None:
        raise ValueError(f"No se encontró la fila de encabezado en el archivo {file_path}")

    return header_row

def read_excel_with_variable_header(file_path):
    header_row = find_header_row(file_path)

    df = pd.read_excel(file_path, header=header_row)

    total_row_index = df[df.iloc[:, 0].str.contains("TOTAL", case=False, na=False)].index.min()

    if total_row_index is not None:
        df = df.iloc[:total_row_index]

    return df

## Read Files

In [5]:
folders = ["Terrorismo","Violencia_Intrafamiliar"]

dataframes_by_folder = {}

def folder_read(folders):
  for folder in folders:
      file_paths = glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xlsx')) + glob.glob(os.path.join(PATH_FILE+f'/{folder}/','*.xls'))
      folder_dataframes = []
      for file_path in file_paths:
          try:
              df = read_excel_with_variable_header(file_path)
              folder_dataframes.append(df)
          except ValueError as e:
              print(e)
      dataframes_by_folder[folder] = folder_dataframes

In [6]:
folder_read(folders)

## Terrorismo

In [7]:
desired_columns = ['FECHA', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'AGRUPA_EDAD_PERSONA', 'GENERO', 'ARMAS_MEDIOS', 'CANTIDAD']
def standardize_columns(df):
    column_mapping = {
        'FECHA ': 'FECHA', 'FECHA HECHO': 'FECHA', 'FECHA HECHO ': 'FECHA','FECHA_HECHO':'FECHA',
        'DEPARTAMENTO ': 'DEPARTAMENTO','Departamento':'DEPARTAMENTO',
        'MUNICIPIO ': 'MUNICIPIO','Municipio':'MUNICIPIO',
        'CODIGO DANE': 'CODIGO_DANE', 'CODIGO DANE ': 'CODIGO_DANE',
        '*AGRUPA_EDAD_PERSONA': 'AGRUPA_EDAD_PERSONA', '*AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA', 'AGRUPA EDAD PERSONA': 'AGRUPA_EDAD_PERSONA',
        'GENERO ': 'GENERO',
        'ARMAS_MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMAS MEDIOS': 'ARMAS_MEDIOS', 'ARMA MEDIO ': 'ARMAS_MEDIOS','ARMAS MEDIO':'ARMAS_MEDIOS',
        'CANTIDAD ': 'CANTIDAD'
    }

    df = df.rename(columns=column_mapping)
    df = df.reindex(columns=desired_columns)
    return df

In [8]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Terrorismo']]
terrorismo_df = pd.concat(standardized_dataframes, ignore_index=True)

In [9]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['GRANADA DE MANO','MINA ANTIPERSONA','ARTEFACTO EXPLOSIVO/CARGA DINAMITA','PAQUETE BOMBA','ARTEFACTO INCENDIARIO',
                             'CILINDRO BOMBA','CANTINA BOMBA','CARRO BOMBA','COMBUSTIBLE','GRANADA DE FUSIL','GRANADA DE MORTERO','MOTO BOMBA',
                             'ARMA DE FUEGO','BALON BOMBA','OLLA BOMBA','PETARDO PLANFETARIO','PAPA EXPLOSIVA','LANCHA BOMBA','ROCKET','CARTA BOMBA',
                             'SIN EMPLEO DE ARMAS','CASA BOMBA','PERRO BOMBA']
terrorismo_df["AGRUPA_EDAD_PERSONA"] = terrorismo_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
terrorismo_df["GENERO"] = terrorismo_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
terrorismo_df["ARMAS_MEDIOS"] = terrorismo_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [10]:
def clean_dane_code(data):
  temp = data.copy()
  temp['CODIGO_DANE'] = temp['CODIGO_DANE'].astype(str)
  filtered_df = temp[temp['CODIGO_DANE'].apply(lambda x: x[-1].isdigit() if x else False)]
  filtered_df.loc[:, 'CODIGO_DANE'] = filtered_df['CODIGO_DANE'].astype(float)
  return filtered_df

In [11]:
terrorismo_filtered = clean_dane_code(terrorismo_df)

In [12]:
terrorismo_output = terrorismo_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS"]).sum().reset_index()

In [13]:
terrorismo_output.to_csv(OUTPUT_PATH_FILE+"/terrorismo.csv",index=False)

## Violencia Intrafamiliar

In [14]:
standardized_dataframes = [standardize_columns(df) for df in dataframes_by_folder['Violencia_Intrafamiliar']]
violencia_intrafamiliar_df = pd.concat(standardized_dataframes, ignore_index=True)

In [15]:
agrupa_edad_persona_valid_values = ["ADULTOS","MENORES","ADOLESCENTES"]
genero_valid_values = ["MASCULINO","FEMENINO"]
armas_medios_valid_values = ['ARMA BLANCA / CORTOPUNZANTE','ARMA DE FUEGO','CONTUNDENTES','CORTANTES','CORTOPUNZANTES','PUNZANTES',
                             'SIN EMPLEO DE ARMAS','ESCOPOLAMINA']
violencia_intrafamiliar_df["AGRUPA_EDAD_PERSONA"] = violencia_intrafamiliar_df["AGRUPA_EDAD_PERSONA"].apply(lambda x: x if x in agrupa_edad_persona_valid_values else "NO REPORTADO")
violencia_intrafamiliar_df["GENERO"] = violencia_intrafamiliar_df["GENERO"].apply(lambda x: x if x in genero_valid_values else "NO REPORTADO")
violencia_intrafamiliar_df["ARMAS_MEDIOS"] = violencia_intrafamiliar_df["ARMAS_MEDIOS"].apply(lambda x: x if x in armas_medios_valid_values else "NO REPORTADO")

In [16]:
violencia_filtered = clean_dane_code(violencia_intrafamiliar_df)

In [17]:
violencia_intrafamiliar_output = violencia_filtered.groupby(
    ["FECHA","DEPARTAMENTO","MUNICIPIO","CODIGO_DANE","AGRUPA_EDAD_PERSONA","GENERO","ARMAS_MEDIOS"]).sum().reset_index()

In [18]:
violencia_intrafamiliar_output.to_csv(OUTPUT_PATH_FILE+"/violencia_intrafamiliar.csv",index=False)